In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys

from datetime import datetime

In [2]:
f = lambda x: 20 + np.e - 20 * np.exp(-0.2 * np.sqrt(np.mean(x ** 2, axis=0))) - np.exp(np.mean(np.cos(2 * np.pi * x), axis=0))

In [3]:
class EvolutionStrategy:
    
    def __init__(self, fitness_func, dim=2, pop_size=100):
        self.pop_size = pop_size
        self.dim = dim
        self.parents_size = self.pop_size // 2
        self.sigma_min = 0.00001
        self.tau1 = 1 / np.sqrt(2 * self.dim)
        self.tau2 = 1 / np.sqrt(2 * np.sqrt(self.dim))
        self.fitness = lambda x: -fitness_func(x[:self.dim])
        
        # Initializing population
        genes = np.random.uniform(-30., 30., size=(self.dim, pop_size))
        strat = np.random.uniform(10., 1., size=(self.dim, pop_size))
        self.pop = np.r_[genes, strat]
        
    def parents_selection(self):
        parents_indexes = np.random.choice(self.pop_size, self.parents_size, replace=False)
        parents = self.pop[:, parents_indexes]
        return parents
    
    def crossover(self, parents, lmbd=200):
        children = np.empty((2*self.dim, lmbd))
        for i in range(lmbd):
            parents_indexes = np.random.choice(parents.shape[1], 2, replace=False)
            genes = parents[:self.dim, parents_indexes]
            strat = parents[self.dim:, parents_indexes]
            
            # Discrete crossover for individual / Intermediate crossover for strategies
            # For individual
            crossover_indexes = np.random.choice(2, self.dim)
            child_genes = np.choose(crossover_indexes, genes.T).reshape(-1, 1)
            
            # For strategy
            child_strats = (strat[:, [0]] + strat[:, [1]]) / 2
            
            children[:, [i]] = np.r_[child_genes, child_strats]
            
        return children
    
    def mutation(self, children):
        genes = children[:self.dim, :]
        strat = children[self.dim:, :]
        
        # Uncorrelated mutation
        new_strat = strat * np.exp(self.tau1 * np.random.normal(size=(strat.shape[1]))) * np.exp(self.tau2 * np.random.normal(size=strat.shape))
        new_strat = np.where(new_strat > self.sigma_min, new_strat, self.sigma_min)
        new_genes = genes + new_strat * np.random.normal(size=genes.shape)
        
        return np.r_[new_genes, new_strat]
        
    def run(self, n_gens=10):
        self.populations = [self.pop]
        for gen in range(int(n_gens)):
            parents = self.parents_selection()
            children = self.crossover(parents)
            mutant_children = self.mutation(children)
            gen_fitness = self.fitness(mutant_children[:self.dim, :])
            
            best_indices = np.argsort(gen_fitness)
            best_fitnesses = gen_fitness[best_indices][-self.pop_size:]
            self.pop = mutant_children[:, best_indices[-self.pop_size:]]
            
            self.populations.append(self.pop)
            sys.stdout.write("\rGeneration %i" % (gen+1))
            sys.stdout.flush()
        print(f'\ngen {gen+1}, best fitnesses: {best_fitnesses[-5:]}')

In [4]:
es = EvolutionStrategy(fitness_func=f, dim=30, pop_size=30)

start = datetime.now()
es.run(n_gens=1000)
print(f'Exec Time: {datetime.now() - start}')

Generation 1000
gen 1000, best fitnesses: [-8.86432454e-05 -8.78815549e-05 -8.62866943e-05 -8.37506197e-05
 -8.23223504e-05]
Exec Time: 0:00:20.223965


In [5]:
# Gráfico 3D para problema de duas dimensões

# populations = es.populations

# x = np.linspace(-30., 30., num=1000)
# y = np.linspace(-30., 30., num=1000)
# coords = np.array(np.meshgrid(x, y))

# fig = go.Figure(
#     data=[go.Surface(x=x, y=y, z=f(coords)), go.Surface(x=x, y=y, z=f(coords))],
#     layout=go.Layout(
#         title_text="Ackley Function", hovermode="closest",
#         updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None])])]),
#     frames=[
#         go.Frame(data=[
#             go.Scatter3d(x=pop[0, :], y=pop[1, :], z=f(pop[:2, :]), mode='markers', marker={'color': 'blue'})
#         ]) for pop in populations
#     ]
# )

# fig.update_layout(autosize=False, width=1000, height=1000, margin=dict(l=65, r=50, b=65, t=90))

# fig.show()

In [6]:
populations = es.populations

# Genes
populations[-1][30:, :][1]

array([1.40158082e-05, 2.02738770e-05, 1.00000000e-05, 1.23431642e-05,
       1.00000000e-05, 1.10725354e-05, 1.00572197e-05, 1.29023847e-05,
       1.50463687e-05, 1.15323995e-05, 1.00000000e-05, 1.00000000e-05,
       1.62381503e-05, 1.23880480e-05, 1.00000000e-05, 1.11483849e-05,
       1.05581584e-05, 1.53885867e-05, 1.18206583e-05, 1.00000000e-05,
       1.00000000e-05, 1.25753089e-05, 2.17061628e-05, 1.00000000e-05,
       1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.17440954e-05,
       1.00000000e-05, 1.44478634e-05])

In [7]:
# Strategies
populations[-1][:30, :][1]

array([ 29.24517661,   0.27408312,   8.00642843, -12.66522836,
        23.40487268,  -2.93205471,  28.90441081,  19.58566732,
        13.06296968,   8.58751476, -24.15964954,  24.06921958,
       -19.85042825,  25.83027272,  -9.43524979, -10.78663233,
        26.12983737, -26.16890871,  26.50016992,  -2.10085172,
       -19.98834919,   3.5021078 , -10.80869601,  28.50280168,
         0.07570573, -10.00668093,  11.14879412, -14.19595114,
         4.33466205,  -1.31827987])